In [1]:
import pandas as pd
from problem_formulation import get_model_for_problem_formulation
from ema_workbench.analysis import plotting, plotting_util, pairs_plotting, prim
from ema_workbench import (RealParameter, ScalarOutcome, Constant,
                           Model, MultiprocessingEvaluator, SequentialEvaluator, ema_logging,
                           perform_experiments, Policy, Scenario)
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.em_framework.optimization import (HyperVolume,
                                                     EpsilonProgress)
from ema_workbench import Constraint
from ema_workbench.util.utilities import (save_results, load_results)
import seaborn as sns
import matplotlib.pyplot as plt

In [68]:
def hurwitz(data,column_names,alpha=0):
    results = {}
    for column_name in column_names:
        Max_value = data[column_name].max()
        Min_value = data[column_name].min()
        hurwitz_coef = alpha * Max_value + (1-alpha) * Min_value
        results[column_name] = hurwitz_coef
    return results

def mean_variance(data,column_names):
    results = {}
    for column_name in column_names:
        results[column_name] = data[column_name].mean(), data[column_name].var()
    return results

In [3]:
# define problem formulation
model, functions = get_model_for_problem_formulation(2)

In [32]:
# use lhs sampling to sample uncertainties and run model
ema_logging.log_to_stderr(ema_logging.INFO)

with MultiprocessingEvaluator(model) as evaluator:
    results = evaluator.perform_experiments(scenarios=4,               
                                            policies=3,
                                            uncertainty_sampling='lhs')

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 4 scenarios * 3 policies * 1 model(s) = 12 experiments
[MainProcess/INFO] 1 cases completed
[MainProcess/INFO] 2 cases completed
[MainProcess/INFO] 3 cases completed
[MainProcess/INFO] 4 cases completed
[MainProcess/INFO] 5 cases completed
[MainProcess/INFO] 6 cases completed
[MainProcess/INFO] 7 cases completed
[MainProcess/INFO] 8 cases completed
[MainProcess/INFO] 9 cases completed
[MainProcess/INFO] 10 cases completed
[MainProcess/INFO] 11 cases completed
[MainProcess/INFO] 12 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [46]:
experiments, outcomes = results

In [47]:
outcomes = pd.DataFrame(outcomes)

In [48]:
data = pd.concat([experiments, outcomes], 1)
columns = outcomes.columns.to_list()
columns.append('policy')
data[columns]

,Expected Annual Damage,Dike Investment Costs,RfR Investment Costs,Evacuation Costs,Expected Number of Deaths,Minimum water level full network,policy
0,2.927113e+07,7.624014e+08,9.665000e+08,2647.747314,0.003891,4.418500,1
1,0.000000e+00,7.624014e+08,9.665000e+08,0.000000,0.000000,5.218298,1
2,6.102194e+04,7.624014e+08,9.665000e+08,4.731652,0.000004,4.418500,1
3,7.068142e+06,7.624014e+08,9.665000e+08,141.535654,0.000805,4.418500,1
4,2.221755e+06,7.822169e+08,1.055500e+09,121.650600,0.000392,4.538500,2
5,0.000000e+00,7.822169e+08,1.055500e+09,0.000000,0.000000,5.338298,2
6,0.000000e+00,7.822169e+08,1.055500e+09,0.000000,0.000000,4.538500,2
7,8.019658e+06,7.822169e+08,1.055500e+09,283.687364,0.002171,4.538500,2
8,2.304408e+08,5.183716e+08,5.269000e+08,0.000000,0.238010,4.298500,3
9,1.089697e+07,5.183716e+08,5.269000e+08,0.000000,0.008486,5.098298,3


In [69]:
hurwitz(data, outcomes.columns.to_list(), 0.5)

[1 2 3]


{'Expected Annual Damage': 115220379.76717158,
 'Dike Investment Costs': 650294259.9311185,
 'RfR Investment Costs': 791200000.0,
 'Evacuation Costs': 1323.873656930931,
 'Expected Number of Deaths': 0.11900514237867287,
 'Minimum water level full network': 4.818399104317777}

In [63]:
dataset = data.groupby('policy').apply(lambda x: hurwitz(x, outcomes.columns.to_list(), 0.5))
pd.DataFrame([value for value in dataset])

,Expected Annual Damage,Dike Investment Costs,RfR Investment Costs,Evacuation Costs,Expected Number of Deaths,Minimum water level full network
0,1.463557e+07,7.624014e+08,9.665000e+08,1323.873657,0.001946,4.818399
1,4.009829e+06,7.822169e+08,1.055500e+09,141.843682,0.001085,4.938399
2,1.206689e+08,5.183716e+08,5.269000e+08,0.000000,0.123248,4.698399


In [65]:
dataset1 = data.groupby('policy').apply(lambda x: mean_variance(x, outcomes.columns.to_list()))
pd.DataFrame([value for value in dataset1])

,Expected Annual Damage,Dike Investment Costs,RfR Investment Costs,Evacuation Costs,Expected Number of Deaths,Minimum water level full network
0,"(9100074.186169248, 191838722331045.22)","(762401392.8017794, 0.0)","(966500000.0, 0.0)","(698.5036551405437, 1692997.0476946312)","(0.001174940841086725, 3.4223030039536657e-06)","(4.618449552158889, 0.15991929363415788)"
1,"(2560353.2487835065, 14343156449575.734)","(782216872.3266463, 0.0)","(1055500000.0, 0.0)","(101.33449093212772, 18067.557569024462)","(0.0006406678008824751, 1.074701924582457e-06)","(4.738449552158889, 0.15991929363415788)"
2,"(70177597.3347504, 1.1448520542074234e+16)","(518371647.53559077, 0.0)","(526900000.0, 0.0)","(0.0, 0.0)","(0.0753027541106215, 0.011898853940071832)","(4.498449552158889, 0.15991929363415788)"
